# Lecura de información

In [20]:
import pandas as pd
import os

# --- 1. CONFIGURACIÓN DE RUTA ---
ruta_script = os.getcwd()
# Ajustamos para subir un nivel y entrar a FUENTE
file_path = os.path.join(ruta_script, "..", "FUENTE", "Afiliación 2025.xlsx")
file_path = os.path.normpath(file_path)

print(f"Leyendo archivo: {file_path}")

# --- 2. CARGA DE DATOS (SIN ENCABEZADOS) ---
# Leemos sin header (header=None) para tener control total de la cuadrícula
# Así podemos acceder por coordenadas (fila, columna) como tú querías
df_raw = pd.read_excel(file_path, sheet_name="MARZO", header=None)

# Definimos dónde empieza la data real
# Fila 1 (índice 1) tiene las fechas: "10-Mar", "17-Mar"
# Fila 2 (índice 2) tiene las métricas: "Bolsa", "Nuevos"...
# Fila 3 (índice 3) empiezan los datos: "Aeropuerto Auto..."
fila_fechas = 1
fila_datos_inicio = 3

# Columnas fijas (Izquierda): Afiliación, Solicitud Nuevos, etc.
# Según tu imagen, son las columnas A, B, C, D (índices 0, 1, 2, 3)
cols_fijas = df_raw.iloc[fila_datos_inicio:, 0:4].copy()
cols_fijas.columns = ["Afiliacion", "Ope_Proy", "Solicitud_Nuevos", "Solicitud_Reconectados"]

# --- 3. EL BUCLE MAESTRO (Tu lógica) ---
lista_dataframes = []

# Empezamos en la columna E (índice 4) que es donde arranca la primera fecha
# Y avanzamos de 5 en 5 (porque cada fecha tiene 5 columnas: Bolsa, Nuevos, Total, Congelas, Activas)
col_inicial = 4
ancho_bloque = 5 
total_cols = df_raw.shape[1]

# Bucle: Mientras no nos salgamos del ancho del Excel
for i in range(col_inicial, total_cols, ancho_bloque):
    
    # 1. Detectar la fecha (está en la fila 1, columna i)
    fecha_header = df_raw.iloc[fila_fechas, i]
    
    # Validación: Si la celda de fecha está vacía o dice "Total cumplimiento", paramos
    if pd.isna(fecha_header) or str(fecha_header).strip() == "" or "cumplimiento" in str(fecha_header):
        continue # Saltamos o terminamos si no es un bloque de fecha válido

    print(f"Procesando fecha: {fecha_header}")

    # 2. Recortar el bloque de 5 columnas correspondiente a esta fecha
    # Seleccionamos todas las filas de datos, y las columnas de i hasta i+5
    bloque_datos = df_raw.iloc[fila_datos_inicio:, i : i + ancho_bloque].copy()
    
    # 3. Asignar nombres estándar a estas 5 columnas
    # OJO: Asegúrate que el orden en el Excel sea siempre este
    bloque_datos.columns = ["Bolsa", "Nuevos", "Total", "Congelas", "Activas"]
    
    # 4. Unir con las columnas fijas (Afiliación, Solicitud...)
    # Usamos reset_index para que las filas coincidan al pegar lado a lado
    df_temp = pd.concat([cols_fijas.reset_index(drop=True), bloque_datos.reset_index(drop=True)], axis=1)
    
    # 5. AGREGAR LA COLUMNA FECHA (Tu requerimiento principal)
    df_temp["Fecha_Corte"] = fecha_header
    
    # Guardamos este fragmento
    lista_dataframes.append(df_temp)

# --- 4. UNIFICACIÓN Y LIMPIEZA FINAL ---
df_final = pd.concat(lista_dataframes, ignore_index=True)

# Limpieza de filas basura (Encabezados repetidos, totales, vacíos)
# Borramos si "Afiliacion" está vacío
df_final = df_final.dropna(subset=["Afiliacion"])
# Borramos si "Afiliacion" dice "Total Flota" o "Afiliación" (el título repetido)
df_final = df_final[~df_final["Afiliacion"].astype(str).str.contains("Total|Afiliación|Plazas", case=False, na=False)]

# Ordenamos columnas para que Fecha salga al inicio (opcional)
cols_orden = ["Fecha_Corte", "Afiliacion", "Ope_Proy", "Solicitud_Nuevos", "Solicitud_Reconectados", "Bolsa", "Nuevos", "Total", "Congelas", "Activas"]
# Filtramos solo columnas que existan (por si acaso Ope_Proy estaba vacía)
cols_existentes = [c for c in cols_orden if c in df_final.columns]
df_final = df_final[cols_existentes]

print("-" * 30)
print("RESULTADO FINAL (Primeras 10 filas):")
print(df_final.head(10))

# Exportar
#df_final.to_csv("Data_Vertical_Final.csv", index=False)

Leyendo archivo: c:\Users\Usuario\Desktop\Scripts_automatización\Afiliación_Activación\FUENTE\Afiliación 2025.xlsx
Procesando fecha: 2025-03-10 00:00:00
Procesando fecha: 2025-03-17 00:00:00
Procesando fecha: 2025-03-24 00:00:00
Procesando fecha: 2025-03-31 00:00:00
------------------------------
RESULTADO FINAL (Primeras 10 filas):
Empty DataFrame
Columns: [Fecha_Corte, Afiliacion, Ope_Proy, Solicitud_Nuevos, Solicitud_Reconectados, Bolsa, Nuevos, Total, Congelas, Activas]
Index: []


In [28]:
df_raw

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Afiliación,Ope proy,NaN,2025-03-10 00:00:00,NaN,NaN,NaN,NaN,2025-03-17 00:00:00,...,NaN,NaN,NaN,2025-03-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Solicitud\nNuevos,Solicitud\nReconectados,Bolsa,nuevos,Total,congelas,Activas,Bolsa,...,Total,congelas,Activas,Bolsa,Nuevos,Total,congelas,Activas,Total cumplimiento,% de cumplimiento
3,NaN,Aeropuerto Auto,30,15,16,10,271,34,237,12,...,276,27,249,15,9,278,30,248,34,0.755556
4,NaN,Aeropuerto Van,7,3,NaN,NaN,18,1,17,NaN,...,19,3,16,NaN,NaN,20,2,18,0,0
5,NaN,Total Flota,NaN,55,NaN,NaN,289,35,254,NaN,...,295,30,265,NaN,NaN,298,32,266,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,Solicitud\nNuevos,Solicitud\nReconectados,Bolsa,nuevos,Total,congelas,Activas,Bolsa,...,Total,congelas,Activas,Bolsa,Nuevos,Total,congelas,Activas,Total cumplimiento,% de cumplimiento
8,NaN,Aerolíneas Auto,25,NaN,10,5,145,23,122,9,...,153,24,129,7,5,160,22,138,19,0.76
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# Usamos .iloc[fila, columna]
# Fila 1 (segunda fila de Excel), Columnas 4, 9, 14, 19
lista_fechas = [
    df_raw.iloc[1, 4], 
    df_raw.iloc[1, 9], 
    df_raw.iloc[1, 14], 
    df_raw.iloc[1, 19]
]

lista_fechas = pd.DataFrame(lista_fechas)
lista_fechas.columns=['Fecha']
lista_fechas.head()

,Fecha
0,2025-03-10
1,2025-03-17
2,2025-03-24
3,2025-03-31


In [ ]:

df_unidades_negocio = df_raw.iloc[3:19, 1:2].copy()
df_unidades_negocio.columns = ['Unidad_Negocio']


df_unidades_negocio.dropna(inplace=True)


df_unidades_negocio = df_unidades_negocio[
    ~df_unidades_negocio['Unidad_Negocio'].astype(str).str.contains("Total|flota", case=False)
]

df_unidades_negocio.reset_index(drop=True, inplace=True)

df_unidades_negocio

,Unidad_Negocio
0,Aeropuerto Auto
1,Aeropuerto Van
2,Aerolíneas Auto
3,Ciudad
4,BASICS
5,Plazas
6,Jockey
7,La Marina
8,Bellavista
9,Angamos


In [47]:
df_unidades_negocio.head()

,Unidad_Negocio
0,Aeropuerto Auto
1,Aeropuerto Van
2,Aerolíneas Auto
3,Ciudad
4,BASICS


In [56]:
df_consolidado = []
df_recorte = []
posFecha = 4
for fecha in lista_fechas['Fecha']:
    sem = 1
    for unidad in df_unidades_negocio['Unidad_Negocio']:
        idx = df_raw[df_raw.iloc[:, 1] == unidad].index[0]
        df_aux = [
            fecha, 
            sem,
            df_raw.iloc[idx, 1],         # Nombre Unidad
            df_raw.iloc[idx, 2],         # Atributo 1 (C)
            df_raw.iloc[idx, 3],         # Atributo 2 (D)
            df_raw.iloc[idx, posFecha],     # Bolsa
            df_raw.iloc[idx, posFecha + 1], # Nuevos
            df_raw.iloc[idx, posFecha + 2], # Total
            df_raw.iloc[idx, posFecha + 3], # Congelas
            df_raw.iloc[idx, posFecha + 4]  # Activas
        ]
        df_consolidado.append(df_aux)
        df_aux = []
    posFecha = posFecha + 5
df_consolidado = pd.DataFrame(df_consolidado)
df_consolidado.columns = ['Fecha Semana', 'No Semana', 'Afiliación', 'Solicitud Nuevos',
'Solicitud Reconectados', 'Bolsa', 'Nuevos', 'Total', 'Congelados', 'Activas']
df_consolidado

,Fecha Semana,No Semana,Afiliación,Solicitud Nuevos,Solicitud Reconectados,Bolsa,Nuevos,Total,Congelados,Activas
0,2025-03-10,1,Aeropuerto Auto,30,15,16,10,271,34,237
1,2025-03-10,1,Aeropuerto Van,7,3,NaN,NaN,18,1,17
2,2025-03-10,1,Aerolíneas Auto,25,NaN,10,5,145,23,122
3,2025-03-10,1,Ciudad,50,NaN,10,5,343,66,277
4,2025-03-10,1,BASICS,50,NaN,NaN,0,7,2,5
5,2025-03-10,1,Plazas,Solicitud\nNuevos,Solicitud\nReconectados,Bolsa,nuevos,Total,congelas,Activas
6,2025-03-10,1,Jockey,8,4,0,0,67,7,60
7,2025-03-10,1,La Marina,5,3,NaN,0,25,3,22
8,2025-03-10,1,Bellavista,3,1,NaN,0,37,9,28
9,2025-03-10,1,Angamos,3,1,NaN,0,7,1,6


In [58]:
df_consolidado[df_consolidado['Afiliación']=='Aeropuerto Auto']

,Fecha Semana,No Semana,Afiliación,Solicitud Nuevos,Solicitud Reconectados,Bolsa,Nuevos,Total,Congelados,Activas
0,2025-03-10,1,Aeropuerto Auto,30,15,16,10,271,34,237
10,2025-03-17,1,Aeropuerto Auto,30,15,12,7,270,23,247
20,2025-03-24,1,Aeropuerto Auto,30,15,12,8,276,27,249
30,2025-03-31,1,Aeropuerto Auto,30,15,15,9,278,30,248
